In [12]:
!pip uninstall matplotlib -y
!pip install matplotlib --upgrade



Found existing installation: matplotlib 3.10.6
Uninstalling matplotlib-3.10.6:
  Successfully uninstalled matplotlib-3.10.6
  Using cached matplotlib-3.10.6-cp313-cp313-win_amd64.whl.metadata (11 kB)
Using cached matplotlib-3.10.6-cp313-cp313-win_amd64.whl (8.1 MB)


In [8]:
""" Class 16. Logistic Regression

Machine Learning Algorithms:
1. Supervised Learning 
   Linear Regression 
   Logistic Regression (Task: Classification)
2. Unsupervised Learning
3. Semi-Supervised Learning
4. Reinforcement Learning

Objectives:
1. Define classification
2. Demonstrate why linear regression is not suitable for classification task.
3. Logistic function
4. Demonstrate cost function
5. Apply gradient descent to logistic regression
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utilities.lab_utils_common import dlc, plot_data, draw_vthresh, sigmoid
from utilities.plt_one_addpt_onclick import plt_one_addpt_onclick
from utilities.plt_logistic_loss import  plt_logistic_cost, plt_two_logistic_loss_curves, plt_simple_example
from utilities.plt_logistic_loss import soup_bowl, plt_logistic_squared_error
%matplotlib notebook
plt.style.use(r'C:\Users\Loccha kakko\PyCharmMiscProject\deeplearning.mplstyle')


In [9]:
DATASET_PATH = r'C:\Users\Loccha kakko\PyCharmMiscProject\cancer (1).csv'
cancer_dataset = pd.read_csv(DATASET_PATH)
cancer_dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84358402,M,20.29,14.34,135.1,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.2,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN
4,844359,M,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.1127,0.07400,...,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,NaN


In [23]:
cancer_dataset['diagnosis'].value_counts()

diagnosis
B    357
M    116
Name: count, dtype: int64

In [10]:
""" Classification: 
Identify the class (target/label, y) that an entity (features, x) belongs to. 
Image => Cat/Dog/Horse
"""

cancer_dataset = cancer_dataset[['area_mean', 'diagnosis']]
cancer_dataset.head()

,area_mean,diagnosis
0,1001.0,M
1,1326.0,M
2,1203.0,M
3,1297.0,M
4,1040.0,M


In [25]:
cancer_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area_mean  473 non-null    float64
 1   diagnosis  473 non-null    object 
dtypes: float64(1), object(1)
memory usage: 7.5+ KB


In [11]:
numerical_cols = cancer_dataset.select_dtypes(include='number').columns
numerical_cols

Index(['area_mean'], dtype='object')

In [12]:
categorical_cols = cancer_dataset.select_dtypes(include='object').columns
categorical_cols

Index(['diagnosis'], dtype='object')

In [13]:
cancer_dataset[numerical_cols]

,area_mean
0,1001.0
1,1326.0
2,1203.0
3,1297.0
4,1040.0
...,...
468,1347.0
469,1479.0
470,1261.0
471,1265.0


In [14]:
cancer_dataset[categorical_cols]

,diagnosis
0,M
1,M
2,M
3,M
4,M
...,...
468,M
469,M
470,M
471,M


In [15]:
min = cancer_dataset[numerical_cols].min()
max = cancer_dataset[numerical_cols].max()

cancer_dataset[numerical_cols] = (cancer_dataset[numerical_cols] - min)/(max - min)
cancer_dataset.head()

,area_mean,diagnosis
0,0.363733,M
1,0.501591,M
2,0.449417,M
3,0.489290,M
4,0.380276,M


In [16]:
cancer_dataset[categorical_cols] = cancer_dataset[categorical_cols].apply(
    lambda col: pd.Categorical(col).codes
)
cancer_dataset.head()

,area_mean,diagnosis
0,0.363733,1
1,0.501591,1
2,0.449417,1
3,0.489290,1
4,0.380276,1


In [17]:
# values = ['B', 'M']
# 'B' => 10
# 'M' => 01
cancer_dataset['diagnosis'].value_counts()

diagnosis
0    357
1    116
Name: count, dtype: int64

In [19]:
import seaborn as sns

sns.relplot(
    data=cancer_dataset,
    x='area_mean',
    y='diagnosis',
    hue='diagnosis',
)

<IPython.core.display.Javascript object>

In [20]:
seed = 142
np.random.seed(seed)

def split_dataset(dataset, training_ratio=0.6, val_ratio=0.2):
    no_of_examples = len(dataset)
    
    random_indices = np.random.permutation(no_of_examples)
    
    train_size = int(no_of_examples * training_ratio)
    val_size = int(no_of_examples * val_ratio)
    
    train_indices = random_indices[:train_size]
    val_indices = random_indices[train_size:train_size + val_size]
    test_indices = random_indices[train_size + val_size:]
    
    train_ds = dataset.iloc[train_indices]
    val_ds = dataset.iloc[val_indices]
    test_ds = dataset.iloc[test_indices]
    
    train_X = np.array(train_ds.iloc[:, :-1])
    train_y = np.array(train_ds.iloc[:, -1])
    val_X = np.array(val_ds.iloc[:, :-1])
    val_y = np.array(val_ds.iloc[:, -1])
    test_X = np.array(test_ds.iloc[:, :-1])
    test_y = np.array(test_ds.iloc[:, -1])
    return train_X, train_y, val_X, val_y, test_X, test_y
    
    
train_X, train_y, val_X, val_y, test_X, test_y = split_dataset(cancer_dataset)

In [22]:
""" linear regression: y = w * x + b 
"""

w = np.zeros(1)
b = 0
plt.close('all') 
__ = plt_one_addpt_onclick(train_X, train_y, w, b, logistic=False)

<IPython.core.display.Javascript object>

AttributeError: 'CheckButtons' object has no attribute 'rectangles'

In [23]:
""" Logistic Regression: 
z = w * x + b
y = g(z)
g is a sigmoid/logistic function.
g(z) = 1 / (1 + e^(-z))
"""

z = np.arange(-10, 11)
g = 1 / (1 + np.exp(-z))

print(f"z={z}\n, g={g}")

z=[-10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3   4   5   6   7
   8   9  10]
, g=[4.54e-05 1.23e-04 3.35e-04 9.11e-04 2.47e-03 6.69e-03 1.80e-02 4.74e-02
 1.19e-01 2.69e-01 5.00e-01 7.31e-01 8.81e-01 9.53e-01 9.82e-01 9.93e-01
 9.98e-01 9.99e-01 1.00e+00 1.00e+00 1.00e+00]


In [24]:
fig,ax = plt.subplots(1,1,figsize=(5,3))
ax.plot(z, g, c="b")
    
ax.set_title("Logistic function")
ax.set_ylabel('Logistic(z)')
ax.set_xlabel('z')
draw_vthresh(ax,0)

<IPython.core.display.Javascript object>

In [25]:
""" ML Algorithm: Logistic Regression"""
def get_cancer_prediction(x, w, b):
    z = w * x + b
    g = 1 / (1 + np.exp(-z))
    return g

In [26]:
plt.close('all') 
__ = plt_one_addpt_onclick(train_X, train_y, w, b, logistic=True)

<IPython.core.display.Javascript object>

AttributeError: 'CheckButtons' object has no attribute 'rectangles'

In [27]:
"""What is concept of selecting cost function for a task?
- Select such a cost function where it is guaranteed that
the gradient descent algorithm is able to minimize the cost function
"""
soup_bowl()

<IPython.core.display.Javascript object>

In [28]:
plt.close('all')
plt_logistic_squared_error(train_X, train_y)
plt.show()

<IPython.core.display.Javascript object>

In [29]:
""" Cost function: Logistic loss / Binary cross entropy loss

log_loss = y_true * log(y_pred) - (1 - y_true) * log(1 - y_pred)

if y_true = 1, log_loss = log(y_pred)
               if y_pred = 1, log_loss = 0
               if y_pred = 0, log_loss = 10
               
if y_true = 0, log_loss = -log(1 - y_pred)
               if y_pred = 1, log_loss = -log(1 - 1) = -log 0 = 10
               if y_pred = 0, log_loss = 0 

y_pred_1, log_loss = 10
y_pred_2, log_loss = 0
y_pred_3, log_loss = 0,
y_pred_4, log_loss = 0

avg. log loss = (10+0+0+0)/4 = 2.5
"""

plt.close('all')
plt_two_logistic_loss_curves()

<IPython.core.display.Javascript object>

In [30]:
plt.close('all')
cst = plt_logistic_cost(train_X, train_y)

<IPython.core.display.Javascript object>

In [68]:
""" Cost function
"""

def cost_function(X, y, w, b):
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        y_pred = get_cancer_prediction(X[i], w, b)   # prediction
        sq_error = (y[i] - y_pred)**2                # squared error
        cost += sq_error

    cost = cost / m   # average
    return cost

In [69]:
log_loss = cost_function(train_X, train_y, w, b)
print(f"Log loss is {log_loss}  (Parameters not learned yet.)")

Log loss is [0.14]  (Parameters not learned yet.)


In [70]:
"""Gradient Descent
- Compute cost (done)
- Compute gradients
- Update parameters
""" 

def compute_gradient(X, y_true, w, b):
    delta = 1e-9
    cost_1 = cost_function(X, y_true, w, b)
    cost_2 = cost_function(X, y_true, w + delta, b)
    cost_3 = cost_function(X, y_true, w, b + delta)
    dw = (cost_2 - cost_1) / delta
    db = (cost_3 - cost_1) / delta
    return dw, db

In [71]:
THRESHOLD = 0.5

def train_model(train_X, train_y, val_X, val_y, epochs=10000, lr=0.001):
    """ Initialize parameters """
    w = np.random.randint(0, 10, 1)
    b = 0

    for epoch in range(epochs):
        """ Compute gradients """
        dw, db = compute_gradient(train_X, train_y, w, b)

        """ Update parameters """
        w = w - lr * dw
        b = b - lr * db

        if epoch % 1000 == 0:
            """ Compute logits / loss """
            loss = cost_function(train_X, train_y, w, b)
            val_loss = cost_function(val_X, val_y, w, b)
            print(f"epoch:{epoch}/{epochs}, loss:{float(loss):.3f}, val_loss:{float(val_loss):0.3f}")

    return w, b

w, b = train_model(train_X, train_y, val_X, val_y)
print("Weight:", w, "bias:", b)


C:\Users\Loccha kakko\AppData\Local\Temp\ipykernel_4240\1014279713.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"epoch:{epoch}/{epochs}, loss:{float(loss):.3f}, val_loss:{float(val_loss):0.3f}")


epoch:0/10000, loss:0.435, val_loss:0.439
epoch:1000/10000, loss:0.393, val_loss:0.401
epoch:2000/10000, loss:0.349, val_loss:0.360
epoch:3000/10000, loss:0.305, val_loss:0.318
epoch:4000/10000, loss:0.264, val_loss:0.278
epoch:5000/10000, loss:0.226, val_loss:0.241
epoch:6000/10000, loss:0.193, val_loss:0.209
epoch:7000/10000, loss:0.165, val_loss:0.181
epoch:8000/10000, loss:0.142, val_loss:0.158
epoch:9000/10000, loss:0.124, val_loss:0.140
Weight: [8.77] bias: [-1.76]


In [74]:
test_loss = cost_function(test_X, test_y, w, b)
print(f"log_loss is {test_loss}")

log_loss is [0.13]


# Evaluation Metrics

In [75]:
""" Evaluation metric: Accuracy
"""

y_pred_train = get_cancer_prediction(train_X, w, b)
y_pred_train = np.where(y_pred_train >= THRESHOLD, 1, 0)

train_acc = 0
for i in range(len(y_pred_train)):
    if y_pred_train[i] == train_y[i]:
        train_acc += 1
train_acc = train_acc / len(y_pred_train) * 100.0

y_pred_test = get_cancer_prediction(test_X, w, b)
y_pred_test = np.where(y_pred_test >= THRESHOLD, 1, 0)

test_acc = 0
for i in range(len(y_pred_test)):
    if y_pred_test[i] == test_y[i]:
        test_acc += 1
train_acc = test_acc / len(y_pred_test) * 100.0

print(f"Train accuracy: {train_acc:0.2f}%")
print(f"test accuracy: {test_acc:0.2f}%")

Train accuracy: 89.58%
test accuracy: 86.00%
